# Data preprocessing

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
from IPython.display import Markdown, display

1) Create file with initial tasks ("initial_tasks.csv)

In [ ]:
# Access folder containing acitivity data
data_folder = "/content/drive/Shareddrives/OSM-ODECO/HOTOSM/activities_output/"
in_drive = True  # True to mount a drive while working in Google Colab
if in_drive:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

In [ ]:
# Read file containing the list of HOT projects to be analised
input_ids_filename = data_folder + "output_archived_projs_selected_ids.csv"
overwrite_if_exists = False
display(Markdown("Reading selected project ids"))
input_data = pd.read_csv(input_ids_filename)

In [ ]:
# Concatenate activities of individual tasks in a single log
activities=pd.DataFrame()
display(Markdown("DOWNLOADING TASK GRIDS FOR THE SELECTED PROJECTS"))
with tqdm(total=len(input_data["projectId"]), unit=" project") as pbar:
  for proj_id in input_data['projectId']:
      input_activities_filename = data_folder + "output_proj_" + str(proj_id) + ".csv"
      input_users_filename = data_folder + "output_users_proj_" + str(proj_id) + ".csv"
      activities = pd.read_csv(input_activities_filename)
      users = pd.read_csv(input_users_filename)
      users = users[["username", "mappingLevel"]]
      # Calculate new taskId
      activities["taskId"]=str(proj_id)+"_"+activities["taskId"].astype(str)
      # Calculate start and end dates
      activities["start"] = pd.to_datetime(activities["actionDate"])
      activities["start"] = np.where(activities["action"] == "STATE_CHANGE", activities["start"]+timedelta(seconds=1), activities["start"])
      activities["complete"] = np.where(activities["action"] == "STATE_CHANGE", activities["start"], activities["start"] + pd.to_timedelta(activities["actionText"], errors="coerce"))
      activities["complete"] = np.where(activities["complete"].isnull(), activities["start"], activities["complete"])
      # Calculate log states
      activities["action"] = np.where(activities["action"] == "STATE_CHANGE", activities["actionText"], activities["action"])
      # Transform task ids into string
      activities["taskId"] = activities["taskId"].apply(str)
      # Keep only the desired columns
      activities = activities[["taskId", "action", "actionBy", "start", "complete"]]
      # Remove ready states
      activities = activities.drop(activities[activities["action"] == "READY"].index)
      # Sort by ascending
      activities = activities.iloc[::-1]
      # Drop duplicates
      activities = activities.drop_duplicates()
      # activities_droped = activities.drop_duplicates(subset=["action", "start", "actionBy"])
      merged = pd.merge(activities, users, left_on="actionBy", right_on="username", how="left")
      activities=activities.append(merged)
      pbar.update(1)

In [ ]:
# Identify first_activities of tasks
activities["start"] = pd.to_datetime(activities["start"])
activities = activities.drop_duplicates()
first_activities=activities.drop_duplicates(subset=["taskId"])
initial_tasks_with_splits=first_activities.sort_values("start").drop_duplicates(subset=["actionBy","start"])

,taskId,action,actionBy,start,complete,username,mappingLevel
0,11875_33,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:45:26.341502+00:00,2021-12-01 04:46:29.933460+00:00,fajarramadhana,BEGINNER
1,11875_72,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:46:37.414302+00:00,2021-12-01 04:47:15.559799+00:00,fajarramadhana,BEGINNER
2,11875_85,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:47:24.839519+00:00,2021-12-01 04:49:46.389017+00:00,fajarramadhana,BEGINNER
3,11875_42,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:49:53.855180+00:00,2021-12-01 04:53:22.263015+00:00,fajarramadhana,BEGINNER
4,11875_42,MAPPED,fajarramadhana,2021-12-01 04:53:23.260095+00:00,2021-12-01 04:53:23.260095+00:00,fajarramadhana,BEGINNER
...,...,...,...,...,...,...,...
1820599,15797_70,LOCKED_FOR_MAPPING,Evan Pearson,2023-11-27 19:10:46.521133+00:00,2023-11-27 19:11:27.473306+00:00,Evan Pearson,BEGINNER
1820600,15797_70,LOCKED_FOR_MAPPING,karabarca29,2023-11-27 19:16:33.278954+00:00,2023-11-27 19:33:55.401159+00:00,karabarca29,BEGINNER
1820601,15797_70,MAPPED,karabarca29,2023-11-27 19:33:56.397329+00:00,2023-11-27 19:33:56.397329+00:00,karabarca29,BEGINNER
1820602,15797_70,LOCKED_FOR_VALIDATION,JuanMelo,2023-11-30 12:26:37.896972+00:00,2023-11-30 14:31:20.822475+00:00,JuanMelo,ADVANCED


In [ ]:
# Identify initial SPLITs and remove duplicates
initial_with_split=initial_tasks_with_splits[initial_tasks_with_splits["action"]=="SPLIT"]
initial_with_split=initial_with_split[initial_with_split.index % 4 == 0]
initial_without_split=initial_tasks_with_splits[initial_tasks_with_splits["action"]!="SPLIT"]
# List of initial activities including unique SPLITs
initial_tasks_with_splits=initial_with_split.taskId.to_list()+initial_without_split.taskId.to_list()

In [ ]:
# Select all the activities of relevant tasks
tasks_with_splits=activities[activities["taskId"].isin(initial_tasks_with_splits)]

,taskId,action,actionBy,start,complete,username,mappingLevel
0,11875_33,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:45:26.341502+00:00,2021-12-01 04:46:29.933460+00:00,fajarramadhana,BEGINNER
1,11875_72,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:46:37.414302+00:00,2021-12-01 04:47:15.559799+00:00,fajarramadhana,BEGINNER
2,11875_85,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:47:24.839519+00:00,2021-12-01 04:49:46.389017+00:00,fajarramadhana,BEGINNER
3,11875_42,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:49:53.855180+00:00,2021-12-01 04:53:22.263015+00:00,fajarramadhana,BEGINNER
4,11875_42,MAPPED,fajarramadhana,2021-12-01 04:53:23.260095+00:00,2021-12-01 04:53:23.260095+00:00,fajarramadhana,BEGINNER
...,...,...,...,...,...,...,...
1820599,15797_70,LOCKED_FOR_MAPPING,Evan Pearson,2023-11-27 19:10:46.521133+00:00,2023-11-27 19:11:27.473306+00:00,Evan Pearson,BEGINNER
1820600,15797_70,LOCKED_FOR_MAPPING,karabarca29,2023-11-27 19:16:33.278954+00:00,2023-11-27 19:33:55.401159+00:00,karabarca29,BEGINNER
1820601,15797_70,MAPPED,karabarca29,2023-11-27 19:33:56.397329+00:00,2023-11-27 19:33:56.397329+00:00,karabarca29,BEGINNER
1820602,15797_70,LOCKED_FOR_VALIDATION,JuanMelo,2023-11-30 12:26:37.896972+00:00,2023-11-30 14:31:20.822475+00:00,JuanMelo,ADVANCED


In [ ]:
# Identify the initial SPLIT of tasks
initial_splits=tasks_with_splits[tasks_with_splits["action"]=="SPLIT"].sort_values("start").drop_duplicates(subset=["taskId"])

,taskId,action,actionBy,start,complete,username,mappingLevel
10896,11893_1247,SPLIT,ALLY SADA,2021-12-06 08:54:19.114617+00:00,2021-12-06 08:54:19.114617+00:00,ALLY SADA,ADVANCED
10964,11893_1251,SPLIT,ALLY SADA,2021-12-06 12:34:21.974559+00:00,2021-12-06 12:34:21.974559+00:00,ALLY SADA,ADVANCED
11616,11893_1373,SPLIT,Seth Mkoma,2021-12-08 13:27:42.880591+00:00,2021-12-08 13:27:42.880591+00:00,Seth Mkoma,ADVANCED
9098,11892_592,SPLIT,cayla_cx,2021-12-09 16:11:18.358458+00:00,2021-12-09 16:11:18.358458+00:00,cayla_cx,BEGINNER
9128,11892_596,SPLIT,HunterH,2021-12-09 16:26:49.923214+00:00,2021-12-09 16:26:49.923214+00:00,HunterH,BEGINNER
...,...,...,...,...,...,...,...
1815264,15689_100,SPLIT,Jasperre,2023-11-09 18:02:49.257273+00:00,2023-11-09 18:02:49.257273+00:00,Jasperre,BEGINNER
1815345,15689_103,SPLIT,koos krijnders,2023-11-10 09:17:51.094956+00:00,2023-11-10 09:17:51.094956+00:00,koos krijnders,ADVANCED
1813436,15655_195,SPLIT,Siem_Hw,2023-11-10 10:49:42.470420+00:00,2023-11-10 10:49:42.470420+00:00,Siem_Hw,ADVANCED
1815441,15689_105,SPLIT,koos krijnders,2023-11-11 19:48:08.564829+00:00,2023-11-11 19:48:08.564829+00:00,koos krijnders,ADVANCED


In [ ]:
# Concatenate the activities of relevant tasks with the initial date of SPLITs in that tasks if present otherwise "2025-12-01 04:46:29.933460+00:00"
initial_tasks_with_splits=tasks_with_splits.merge(initial_splits[["taskId","start"]], on="taskId", how="left").fillna(value=pd.to_datetime('2025-12-01 04:46:29.933460+00:00'))

,taskId,action,actionBy,start_x,complete,username,mappingLevel,start_y
0,11875_33,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:45:26.341502+00:00,2021-12-01 04:46:29.933460+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
1,11875_72,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:46:37.414302+00:00,2021-12-01 04:47:15.559799+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
2,11875_85,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:47:24.839519+00:00,2021-12-01 04:49:46.389017+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
3,11875_42,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:49:53.855180+00:00,2021-12-01 04:53:22.263015+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
4,11875_42,MAPPED,fajarramadhana,2021-12-01 04:53:23.260095+00:00,2021-12-01 04:53:23.260095+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
...,...,...,...,...,...,...,...,...
1381179,15797_70,LOCKED_FOR_MAPPING,Evan Pearson,2023-11-27 19:10:46.521133+00:00,2023-11-27 19:11:27.473306+00:00,Evan Pearson,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381180,15797_70,LOCKED_FOR_MAPPING,karabarca29,2023-11-27 19:16:33.278954+00:00,2023-11-27 19:33:55.401159+00:00,karabarca29,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381181,15797_70,MAPPED,karabarca29,2023-11-27 19:33:56.397329+00:00,2023-11-27 19:33:56.397329+00:00,karabarca29,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381182,15797_70,LOCKED_FOR_VALIDATION,JuanMelo,2023-11-30 12:26:37.896972+00:00,2023-11-30 14:31:20.822475+00:00,JuanMelo,ADVANCED,2025-12-01 04:46:29.933460+00:00


In [ ]:
# Remove all the activities happening after observing a split in relevant tasks
initial_tasks_with_splits['start_y'] = pd.to_datetime(initial_tasks_with_splits["start_y"], utc=True)
initial_tasks_with_splits['start_x'] = pd.to_datetime(initial_tasks_with_splits["start_x"], utc=True)
initial_tasks_clean=initial_tasks_with_splits[initial_tasks_with_splits["start_x"]<=initial_tasks_with_splits["start_y"]]

,taskId,action,actionBy,start_x,complete,username,mappingLevel,start_y
0,11875_33,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:45:26.341502+00:00,2021-12-01 04:46:29.933460+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
1,11875_72,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:46:37.414302+00:00,2021-12-01 04:47:15.559799+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
2,11875_85,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:47:24.839519+00:00,2021-12-01 04:49:46.389017+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
3,11875_42,LOCKED_FOR_MAPPING,fajarramadhana,2021-12-01 04:49:53.855180+00:00,2021-12-01 04:53:22.263015+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
4,11875_42,MAPPED,fajarramadhana,2021-12-01 04:53:23.260095+00:00,2021-12-01 04:53:23.260095+00:00,fajarramadhana,BEGINNER,2025-12-01 04:46:29.933460+00:00
...,...,...,...,...,...,...,...,...
1381179,15797_70,LOCKED_FOR_MAPPING,Evan Pearson,2023-11-27 19:10:46.521133+00:00,2023-11-27 19:11:27.473306+00:00,Evan Pearson,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381180,15797_70,LOCKED_FOR_MAPPING,karabarca29,2023-11-27 19:16:33.278954+00:00,2023-11-27 19:33:55.401159+00:00,karabarca29,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381181,15797_70,MAPPED,karabarca29,2023-11-27 19:33:56.397329+00:00,2023-11-27 19:33:56.397329+00:00,karabarca29,BEGINNER,2025-12-01 04:46:29.933460+00:00
1381182,15797_70,LOCKED_FOR_VALIDATION,JuanMelo,2023-11-30 12:26:37.896972+00:00,2023-11-30 14:31:20.822475+00:00,JuanMelo,ADVANCED,2025-12-01 04:46:29.933460+00:00


In [ ]:
# Create file with the log for initial tasks ("initial_tasks.csv")
initial_tasks_clean.columns=['taskId', 'action', 'actionBy', 'start', 'complete', 'username','mappingLevel', 'start_y']
initial_tasks_clean=initial_tasks_clean[['taskId', 'action', 'actionBy', 'start', 'complete','mappingLevel']]
initial_tasks_clean.to_csv("initial_tasks.csv", index=False)

2) Create file with contributor profile ("contributors.csv")

In [ ]:
# Concatenate contributors of selected projects
contributors=pd.DataFrame()
display(Markdown("DOWNLOADING TASK GRIDS FOR THE SELECTED PROJECTS"))
with tqdm(total=len(input_data["projectId"]), unit=" project") as pbar:
  for index, row in input_data.iterrows():
      input_users_filename = data_folder + "output_users_proj_" + str(row["projectId"]) + ".csv"
      users = pd.read_csv(input_users_filename)
      users = users[["username", "mappingLevel"]]
      contributors=contributors.append(users)
      pbar.update(1)
contributors=contributors.drop_duplicates()

In [ ]:
# Create file with contributors profile ("contributors.csv")
contributors.to_csv("contributors.csv", index=False)

3) Create file for regression analysis with task density ("regression.csv.csv)

In [ ]:
# Read file with densities per tasks
densities=pd.read_csv("output_densities.csv")
densities["taskId"]=densities["projId"].astype(str)+"_"+densities["taskId"].astype(str)

In [ ]:
# Dummy encoding por activity presence
activities['splits'] = np.where(activities['action']=='SPLIT', 1, 0)
activities['invalidations'] = np.where(activities['action']=='INVALIDATED', 1, 0)
activities['locked_for_mappings'] = np.where(activities['action']=='LOCKED_FOR_MAPPING', 1, 0)
activities['mapped'] = np.where(activities['action']=='MAPPED', 1, 0)
activities['locked_for_validation'] = np.where(activities['action']=='LOCKED_FOR_VALIDATION', 1, 0)
activities['validations'] = np.where(activities['action']=='VALIDATED', 1, 0)
activities['auto_unlocked_for_mapping'] = np.where(activities['action']=='AUTO_UNLOCKED_FOR_MAPPING', 1, 0)
activities['auto_unlocked_for_validation'] = np.where(activities['action']=='AUTO_UNLOCKED_FOR_VALIDATION', 1, 0)
activities['badimagery'] = np.where(activities['action']=='BADIMAGERY', 1, 0)
activities['extended_for_mapping'] = np.where(activities['action']=='EXTENDED_FOR_MAPPING', 1, 0)

In [ ]:
# Group by TaskId to count the number or ocurrences of each activity per tasks and adding density per task
regression=activities[["taskId","splits","invalidations","locked_for_mappings","mapped","locked_for_validation","validations","auto_unlocked_for_mapping","auto_unlocked_for_validation","badimagery","extended_for_mapping"]].groupby(["taskId"]).sum().reset_index().merge(densities, on="taskId", how="left")
regression['projectId'] = regression['taskId'].str.split('_').str[0]
projects['projectId'] = projects['projectId'].astype(str)
regression=regression.merge(projects[["projectId","difficulty","priority"]], on="projectId", how="left")
regression["splits1"]= np.where(regression.splits==0,0,1)
regression["invalidations1"]= np.where(regression.invalidations==0,0,1)
regression["locked_for_mappings1"]= np.where(regression.locked_for_mappings==0,0,1)
regression["mapped1"]= np.where(regression.mapped==0,0,1)
regression["locked_for_validation1"]= np.where(regression.locked_for_validation==0,0,1)
regression["validations1"]= np.where(regression.validations==0,0,1)
regression["auto_unlocked_for_mapping1"]= np.where(regression.auto_unlocked_for_mapping==0,0,1)
regression["auto_unlocked_for_validation1"]= np.where(regression.auto_unlocked_for_validation==0,0,1)
regression["badimagery1"]= np.where(regression.badimagery==0,0,1)
regression["extended_for_mapping1"]= np.where(regression.extended_for_mapping==0,0,1)
regression['projId'] = regression['projId'].apply(str)
regression

In [ ]:
# Create file with regression data ("regression.csv")
regression.to_csv("regression.csv",index=False)